In [257]:
import pandas as pd
import seaborn as sb

In [258]:
bruggen = pd.read_csv('bruggen.csv')

In [259]:
bruggen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141 entries, 0 to 140
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gemeente           141 non-null    object 
 1   gewest             141 non-null    object 
 2   provincie          141 non-null    object 
 3   brugnaam           141 non-null    object 
 4   latitude           138 non-null    float64
 5   longitude          138 non-null    float64
 6   probleem           131 non-null    object 
 7   oorzaak            131 non-null    object 
 8   werken_gepland     138 non-null    object 
 9   start werken       138 non-null    object 
 10  bezocht            78 non-null     object 
 11  opmerkingen        29 non-null     object 
 12  beelden            20 non-null     object 
 13  beheerder          141 non-null    object 
 14  hoogproblematisch  141 non-null    object 
 15  lijst_sinds        141 non-null    object 
dtypes: float64(2), object(14)


In [260]:
bruggen = bruggen.rename(columns = {'start werken':'start_werken'})

In [261]:
bruggen.head()

,gemeente,gewest,provincie,brugnaam,latitude,longitude,probleem,oorzaak,werken_gepland,start_werken,bezocht,opmerkingen,beelden,beheerder,hoogproblematisch,lijst_sinds
0,Wommelgem,Vlaanderen,Antwerpen,B5 over de E313 in de Wijnegemsteenweg,51.212246,4.527518,Vallende brokstukken. Afgesloten voor alle gem...,brokkelt,Nee,onbekend,Nee,NaN,NaN,AWV,ja,2007
1,Deurne,Vlaanderen,Antwerpen,Brug 2 in de E34 over de Sterckshoflei,51.210954,4.464007,Jaarlijkse opvolging en waterpassing,technisch,Ja,2021,Nee,NaN,NaN,AWV,ja,2003
2,Arendonk,Vlaanderen,Antwerpen,Brug 5 in de N139 over kanaal in Arendonk,51.338964,5.104604,Renovatieplannen in opmaak,onbekend,Ja,2020,Nee,NaN,NaN,AWV,ja,2011
3,Deurne,Vlaanderen,Antwerpen,Brug 1 in de E34 over de Cornelissenlaan,51.214129,4.459276,"3-maandelijke inspectie, 6-maandelijkse waterp...",technisch,Ja,2022,Nee,NaN,NaN,AWV,ja,2003
4,Zaventem,Vlaanderen,Vlaams-Brabant,Brug B16 in Hector Henneaulaan over de R0 en R22,50.880841,4.460796,Moet vernieuwd worden bij verbreden R0,groter plan,Ja,2022,Nee,NaN,NaN,AWV,ja,2016


## Hoeveel hoogproblematische bruggen zijn er?

In [262]:
len(bruggen[bruggen.hoogproblematisch == 'ja'].index)

56

In [263]:
len(bruggen.index)

141

In [264]:
56/141*100

39.71631205673759

Van alle bruggen die op de lijst hebben staan is 40 procent hoogproblematisch. Hoogproblematisch is een term die de beheerders zelf hebben toegekend aan de staat van bruggen, dit hoeft dus niet consistent te zijn over alle beheerders heen. Wat bij de ene beheerder problematisch heet, hoeft dat bij de ander niet te zijn.

## Welke gemeenten heeft de meeste bruggen die gerenoveerd moeten worden/de meeste probleembruggen?

In [265]:
bruggen['totaal_gemeente'] = bruggen.groupby('gemeente')['brugnaam'].transform('count')

In [266]:
probleem = bruggen.groupby(['gemeente', 'hoogproblematisch']).size().reset_index()

In [267]:
probleem = probleem.rename(columns={0: 'totaal_probleem'})

In [268]:
probleem = probleem[probleem.hoogproblematisch == 'ja']

In [269]:
probleem = probleem[['gemeente', 'totaal_probleem']]

In [270]:
gemeentes = bruggen.merge(probleem, on='gemeente')

In [271]:
gemeentes['percentage'] = gemeentes['totaal_probleem'] / gemeentes['totaal_gemeente'] * 100

In [272]:
gemeentes['percentage'] = gemeentes.percentage.astype(int)

In [273]:
gemeentes.groupby('gemeente')['totaal_gemeente'].max().sort_values(ascending=False).head(10)

gemeente
Gent            24
Anderlecht      20
Brussel-Stad    14
Oudergem         4
Schaarbeek       4
Vorst            3
Mechelen         3
Zaventem         3
Elsene           3
Grimbergen       2
Name: totaal_gemeente, dtype: int64

In [274]:
gemeentes.groupby('gemeente')['totaal_probleem'].max().sort_values(ascending=False).head(10)

gemeente
Gent            14
Zaventem         3
Brussel-Stad     3
Anderlecht       2
Halle            2
Grimbergen       2
Oudergem         2
Deurne           2
Kortrijk         2
Overrijse        2
Name: totaal_probleem, dtype: int64

In Gent moeten de meeste bruggen gerenoveerd worden en staan ook in absolute zin de meeste hoogproblematische bruggen. Daarna volgt Zaventem, met 3 hoogproblematische bruggen, net als Brussel-Stad. Anderlecht staat op de tweede plek met de meeste bruggen die gerenoveerd moeten worden, met daarna Brussel-Stad. Het moet hier opgemerkt worden dat we voor het Brussels Gewest op de hoogte zijn van alle bruggen die gerenoveerd moeten worden, terwijl we van bijvoorbeeld AWV en VWW alleen weten welke bruggen op het lijstje met meest problematische bruggen staan. Voor Infrabel hebben we zelfs nog minder informatie, en weten we dat onderscheid helemaal niet. 

In [275]:
gemeentes[gemeentes.percentage == 100].gemeente.unique()

array(['Wommelgem', 'Deurne', 'Arendonk', 'Zaventem', 'Overrijse',
       'Halle', 'Grimbergen', 'Wetteren', 'Erpe-Mere', 'Waregem',
       'Oostende', 'Nieuwpoort', 'Kortrijk', 'Zwevegem', 'Aalst',
       'Sint-Laureins', 'Wielsbeke', 'Deinze', 'Voeren', 'Galmaarden'],
      dtype=object)

In [276]:
gemeentes.percentage.unique()

array([100,  58,  50,  33,  25,  21,  10])

In [277]:
gemeentes[gemeentes.percentage == 58].gemeente.unique()

array(['Gent'], dtype=object)

In [278]:
gemeentes[gemeentes.percentage == 50].gemeente.unique()

array(['Roeselare', 'Merelbeke', 'Brugge', 'Oudergem',
       'Sint-Agatha-Berchem'], dtype=object)

Er zijn dus een aantal gemeenten waar alle bruggen die gerenoveerd moeten worden ook hoogproblematische bruggen zijn. Dat zijn Wommelgem, Deurne, Arendonk, Zaventem, Overijse, Halle, Grimbergen, Wetteren, Erpe-Mere, Waregem, Oostende, Nieuwpoort, Kortrijk, Zwevegem, Aalst, Sint-Laureins, Wielsbeke, Deinze, Voeren en Galmaarden. Dan zijn er nog gemeenten waar minstens de helft van de bruggen die gerenoveerd moeten worden hoogproblematisch zijn, namelijk Gent (58 procent), Roeselare, Merelbeke, Brugge, Oudergem en Sint-Agatha-Berchem.

In [334]:
gemeentes[gemeentes.percentage == 50]

,gemeente,gewest,provincie,brugnaam,latitude,longitude,probleem,oorzaak,werken_gepland,start_werken,bezocht,opmerkingen,beelden,beheerder,hoogproblematisch,lijst_sinds,totaal_gemeente,totaal_probleem,percentage
40,Roeselare,Vlaanderen,West-Vlaanderen,Brug B50 N36 Roeselare-Kerkhove in Roeselare (...,50.918747,3.176617,Schade na ongeval,ongeval,Ja,onbekend,Ja,Geen zichtbare schade,Nee,AWV,ja,2018,2,1,50
41,Roeselare,Vlaanderen,West-Vlaanderen,Spiraalbrug (fiets en wandelbrug) over Koning ...,50.944055,3.132034,corrosie,corrosie,Ja,Deze legislatuur,Nee,De nodige budgetten werden voorzien in de meer...,NaN,Gemeente,onbekend,onbekend,2,1,50
48,Merelbeke,Vlaanderen,Oost-Vlaanderen,Vandepittebrug over de Ringvaart in Merelbeke,51.007778,3.751424,"Moet gerenoveerd worden, maar niet dringend",onbekend,Nee,onbekend,Ja,Veel verroest metaal te zien. Via de kant van ...,Foto en video,Vlaamse Waterweg,ja,2017,2,1,50
49,Merelbeke,Vlaanderen,Oost-Vlaanderen,Brug over de Ringvaart en R4 bij Fraterstraat,51.005171,3.760860,slijtage,verouderd,Ja,2021,Nee,NaN,NaN,Gemeente,onbekend,onbekend,2,1,50
53,Brugge,Vlaanderen,West-Vlaanderen,Fietsbrug aan de Kleine Boudewijnbrug,51.222415,3.223436,Bepaalde delen worden gestraald om onderzoek m...,onbekend,Ja,onbekend,Nee,NaN,NaN,Vlaamse Waterweg,ja,2018,2,1,50
54,Brugge,Vlaanderen,West-Vlaanderen,Fietersbrug aan de Smedenpoort,51.204723,3.212829,corrosie,corrosie,Ja,Deskundige aangesteld,Nee,NaN,NaN,Gemeente,onbekend,onbekend,2,1,50
64,Oudergem,Brussels Gewest,Brussel,KW 009 Beaulieu,50.815109,4.408260,minder belangrijke gebreken aan dragende struc...,onbekend,Ja,Bezig,NaN,NaN,NaN,Brussels Gewest,nee,onbekend,4,2,50
65,Oudergem,Brussels Gewest,Brussel,KW 011 Viaduct Watermaal,50.814639,4.417221,verborgen gebreken mogelijk maar bepaalde dele...,onbekend,onbekend,onbekend,NaN,NaN,NaN,Brussels Gewest,nee,onbekend,4,2,50
66,Oudergem,Brussels Gewest,Brussel,KW 012 Herrmann-Debroux,50.813182,4.435382,"evolutieve gebreken, maar wordt momenteel hers...",onbekend,Ja,Bezig,NaN,NaN,NaN,Brussels Gewest,ja,onbekend,4,2,50
67,Oudergem,Brussels Gewest,Brussel,KW 013 Drie Fonteinen,50.808044,4.447835,"evolutieve gebreken, maar wordt momenteel hers...",onbekend,Ja,Bezig,NaN,NaN,NaN,Brussels Gewest,ja,onbekend,4,2,50


In [279]:
bruggen[bruggen.gemeente == 'Gent'].oorzaak.value_counts()

onbekend        19
groter plan      2
brokkelt         1
groter plan      1
ongeval          1
Name: oorzaak, dtype: int64

In [280]:
bruggen[bruggen.gemeente == 'Gent'].werken_gepland.value_counts()

Ja     21
Nee     3
Name: werken_gepland, dtype: int64

In [281]:
bruggen[bruggen.gemeente == 'Gent'].start_werken.value_counts()

onbekend    9
2021        4
2020        4
2019        2
2024        2
10-2020     1
2023        1
04-2020     1
Name: start_werken, dtype: int64

In [282]:
bruggen[bruggen.gemeente == 'Anderlecht'].oorzaak.value_counts()

onbekend    20
Name: oorzaak, dtype: int64

In [283]:
bruggen[bruggen.gemeente == 'Anderlecht'].werken_gepland.value_counts()

onbekend    17
Ja           3
Name: werken_gepland, dtype: int64

In [284]:
bruggen[bruggen.gemeente == 'Anderlecht'].start_werken.value_counts()

onbekend    17
Bezig        3
Name: start_werken, dtype: int64

In de gemeente Gent is de oorzaak van de problematiek van de meeste bruggen onbekend. In Anderlecht geldt dit voor alle bruggen. Voor de meeste bruggen in Gent is renovatie gepland - in Anderlecht is dit voor de meeste bruggen onbekend. Die bruggen die wel gerenoveerd zijn, zijn al bezig. In Gent zouden zes bruggen nog dit jaar gerenoveerd worden, twee zouden al gerenoveerd moeten zijn vorig jaar, voor de vorige bruggen is een latere datum voorzien of is het nog niet bekend.

## In welke provincies staan de meeste bruggen die gerenoveerd moeten worden / hoogproblematische bruggen?

In [285]:
bruggen.provincie.value_counts()

Brussel            55
Oost-Vlaanderen    33
West-Vlaanderen    18
Vlaams-Brabant     17
Antwerpen          15
Limburg             3
Name: provincie, dtype: int64

In [286]:
bruggen.gewest.value_counts()

Vlaanderen         86
Brussels Gewest    55
Name: gewest, dtype: int64

De meeste bruggen moeten worden gerenoveerd in Brussel, daarna in Oost-Vlaanderen, daarna in West-Vlaanderen. Hierbij moet weer aangetekend worden dat we alleen voor het Brussels gewest op de hoogte zijn van bruggen die wel gerenoveerd moeten worden, maar die niet hoogproblematisch zijn. De minste bruggen moeten gerenoveerd worden in Limburg en de provincie Antwerpen. Er staan meer te renoveren bruggen in het Vlaams Gewest, maar omdat we niet van de gewestelijke instellingen weten hoeveel 'gewone' bruggen die gerenoveerd moeten worden zij hebben, is de kans groot dat dit aantal nog hoger ligt. Bovendien zijn er in Vlaanderen ook simpelweg meer bruggen dan in Brussel.

In [287]:
bruggen[bruggen.hoogproblematisch == 'ja'].provincie.value_counts()

Oost-Vlaanderen    20
Brussel            11
Vlaams-Brabant     10
West-Vlaanderen     9
Antwerpen           5
Limburg             1
Name: provincie, dtype: int64

In [288]:
bruggen[bruggen.hoogproblematisch == 'ja'].gewest.value_counts()

Vlaanderen         45
Brussels Gewest    11
Name: gewest, dtype: int64

De meeste hoogproblematische bruggen staan in Oost-Vlaanderen, daarna in Brussel. In het Vlaams gewest staan beduidend meer hoogproblematische bruggen dan in Brussel. Hoe zit dat in relatieve zin?

In [324]:
11/55 * 100

20.0

In [325]:
20/33 * 100

60.60606060606061

Kortom: in Brussel moeten misschien meer bruggen gerenoveerd worden, maar uiteindelijk staan er naar verhouding en in absolute zin meer problematische bruggen in Oost-Vlaanderen. Hier is wel weer de aantekening te maken dat we niet precies weten hoeveel laagproblematische bruggen er gerenoveerd zouden moeten worden in Vlaanderen, omdat we die informatie van de gewestelijke instellingen niet hebben.

## Hoe zit het per beheerder?
We kunnen ook nog bekijken hoe dat zit met de totale bruggen die in beheer zijn. Daar hebben we informatie over van het Brussels Gewest, VWW en AWV tezamen. Van Infrabel weten we wel hoeveel bruggen zij beheren, maar niet hoeveel daarvan precies gerenoveerd moeten worden danwel problematisch zijn. Voor de gemeenten weten we wel hoeveel bruggen gerenoveerd moeten worden, maar niet exact hoeveel daarvan precies problematisch genoemd zouden moeten worden en hoeveel bruggen zij precies in totaal in bezit hebben.

In [291]:
52/88 * 100

59.09090909090909

In [292]:
11/88 * 100

12.5

In Brussel moet dus zo'n 60 procent van de bruggen in beheer gerenoveerd worden, waarvan 12,5% problematisch is.

In [293]:
42/2618 * 100

1.6042780748663104

Onder het beheer van het Vlaams Gewest is zo'n 1,6 procent van de bruggen hoogproblematisch. Meer weten we niet, omdat we niet weten hoeveel bruggen er precies gerenoveerd moeten worden die niet hoogproblematisch zijn. De uitsplitsing tussen AWV/VWW is ongeveer 800 bruggen voor VWW, dus ongeveer 2 procent van de bruggen van VWW is problematisch, tegenover 1,2 procent van de AWV. In z'n totaliteit (voor de bekende cijfers) betekent dat ongeveer 2 procent van de bruggen hoogproblematisch is.

In [294]:
bruggen.beheerder.unique()

array(['AWV', 'Vlaamse Waterweg', 'Gemeente', 'Brussels Gewest',
       'Infrabel'], dtype=object)

In [295]:
bruggen[bruggen.beheerder == 'AWV'].gemeente.value_counts()

Gent          7
Zaventem      3
Grimbergen    2
Overrijse     2
Deurne        2
Wommelgem     1
Erpe-Mere     1
Halle         1
Oostende      1
Wetteren      1
Waregem       1
Nieuwpoort    1
Arendonk      1
Roeselare     1
Name: gemeente, dtype: int64

In [296]:
bruggen[bruggen.beheerder == 'Vlaamse Waterweg'].gemeente.value_counts()

Gent             7
Kortrijk         2
Wielsbeke        1
Deinze           1
Zwevegem         1
Aalst            1
Halle            1
Brugge           1
Sint-Laureins    1
Merelbeke        1
Name: gemeente, dtype: int64

In [327]:
bruggen[bruggen.beheerder == 'Infrabel'].provincie.value_counts()

Antwerpen    8
Brussel      2
Name: provincie, dtype: int64

Van wat we weten van zowel de VWW als de AWV hebben zij de meeste te renoveren bruggen in Gent. Voor Infrabel zijn de meeste te renoveren bruggen te vinden in de provincie Antwerpen.

## Hoe lang staan bruggen op de lijst voor ze gerenoveerd worden?

Dit weten we alleen voor de bruggen onder het beheer van het Vlaams Gewest. Daarvoor kunnen we kijken naar de column "lijst_sinds".

In [298]:
vlaams = bruggen[bruggen.lijst_sinds != 'onbekend']

In [299]:
vlaams['lijst_sinds'] = vlaams.lijst_sinds.astype(int)

/Users/fmeijer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [300]:
len(vlaams[vlaams.lijst_sinds < 2000].index)

5

In [301]:
len(vlaams.index)

42

In [302]:
5 / 42 * 100

11.904761904761903

Er staan 5 bruggen op de lijst van het Vlaamse Gewest die al vóór 2000 op de lijst stonden en er dus al meer dan 20 jaar op staan.

In [303]:
vlaams[vlaams.lijst_sinds < 2000].gemeente.value_counts()

Gent        4
Wetteren    1
Name: gemeente, dtype: int64

De meeste van deze bruggen staan in Gent. Er staat er één in Wetteren.

In [304]:
len(vlaams[vlaams.lijst_sinds <= 2015].index)

21

In [305]:
21 / 42 * 100

50.0

De helft van de bruggen van het Vlaams Gewest - en dat zijn allemaal hoogproblematische bruggen - staat al minstens 5 jaar op de lijst. 

In [306]:
vlaams[vlaams.lijst_sinds <= 2015].gemeente.value_counts()

Gent         9
Zaventem     2
Kortrijk     2
Deurne       2
Wommelgem    1
Erpe-Mere    1
Waregem      1
Halle        1
Arendonk     1
Wetteren     1
Name: gemeente, dtype: int64

Het is duidelijk dat er vrij veel bruggen in Gent al een geruime tijd op de lijst staan om gerenoveerd te worden.

In [307]:
vlaams[vlaams.lijst_sinds <= 2015].provincie.value_counts()

Oost-Vlaanderen    11
Antwerpen           4
West-Vlaanderen     3
Vlaams-Brabant      3
Name: provincie, dtype: int64

## Welke problemen spelen er bij de bruggen?

In [308]:
bruggen = bruggen.replace('groter plan ', 'groter plan')

In [309]:
bruggen.oorzaak.value_counts()

onbekend       91
technisch      11
verouderd      10
groter plan     6
ongeval         4
brokkelt        2
corrosie        2
doorbuiging     2
betonrot        2
verzakking      1
Name: oorzaak, dtype: int64

Van de meeste bruggen weten we niet precies wat de oorzaak is dat ze gerenoveerd moeten worden. Daarna worden de meeste bruggen gerenoveerd als onderdeel van een technisch probleem, of omdat ze verouderd zijn. Daarna vanwege groter plan, of vanwege een ongeval.

In [310]:
bruggen[bruggen.oorzaak == 'onbekend'].provincie.value_counts()

Brussel            53
Oost-Vlaanderen    25
West-Vlaanderen    10
Vlaams-Brabant      2
Antwerpen           1
Name: provincie, dtype: int64

In [311]:
bruggen.provincie.value_counts()

Brussel            55
Oost-Vlaanderen    33
West-Vlaanderen    18
Vlaams-Brabant     17
Antwerpen          15
Limburg             3
Name: provincie, dtype: int64

De bruggen met onbekende oorzaak staan grotendeels in Brussel. Ook in Oost-Vlaanderen staan veel bruggen waarvan niet precies duidelijk is waarom ze gerenoveerd moeten worden. In Brussel geeft het gewest alleen aan of er bijvoorbeeld 'evolutieve gebreken' zijn, of 'gebreken aan de draagconstructie'. Waar deze gebreken door worden veroorzaakt, wordt niet gemeld.

In [330]:
bruggen[bruggen.oorzaak == 'technisch']

,gemeente,gewest,provincie,brugnaam,latitude,longitude,probleem,oorzaak,werken_gepland,start_werken,bezocht,opmerkingen,beelden,beheerder,hoogproblematisch,lijst_sinds,totaal_gemeente
1,Deurne,Vlaanderen,Antwerpen,Brug 2 in de E34 over de Sterckshoflei,51.210954,4.464007,Jaarlijkse opvolging en waterpassing,technisch,Ja,2021,Nee,NaN,NaN,AWV,ja,2003,2
3,Deurne,Vlaanderen,Antwerpen,Brug 1 in de E34 over de Cornelissenlaan,51.214129,4.459276,"3-maandelijke inspectie, 6-maandelijkse waterp...",technisch,Ja,2022,Nee,NaN,NaN,AWV,ja,2003,2
52,Wijnegem,Vlaanderen,Antwerpen,Brug op de N12 bij het Albertkanaal,51.234017,4.515477,doorvaarthoogte is te laag en wordt opgehoogd,technisch,Ja,2023,Nee,NaN,NaN,Gemeente,nee,onbekend,1
55,De Pinte,Vlaanderen,Oost-Vlaanderen,Baron de Gieylaan,51.000933,3.660068,nieuwe asfaltlaag,technisch,Ja,onbekend,Nee,NaN,NaN,Gemeente,nee,onbekend,2
56,De Pinte,Vlaanderen,Oost-Vlaanderen,Pont-Noord over de E17,50.988343,3.702963,fietspaden voorzien,technisch,Ja,onbekend,NaN,NaN,NaN,Gemeente,nee,onbekend,2
61,Kuurne,Vlaanderen,West-Vlaanderen,Kuurnebrug over de Leie,50.850626,3.300125,brug ophogen ihkv Europees project,technisch,Ja,2021,Nee,NaN,NaN,Gemeente,nee,onbekend,3
63,Kuurne,Vlaanderen,West-Vlaanderen,Fiets- en wandelbrug over de Heulebeek bij de ...,50.847096,3.272605,extra inzetten op verbindingswegen voor trage ...,technisch,Ja,2022,Nee,NaN,NaN,Gemeente,nee,onbekend,3
65,Schoten,Vlaanderen,Antwerpen,Hoogmolenbrug,51.239972,4.490745,In het kader van het verbeteren van de bereikb...,technisch,Ja,Bezig,Nee,NaN,NaN,Gemeente,nee,onbekend,1
70,Beersel,Vlaanderen,Vlaams-Brabant,Brug over de Zenne in de Zennestraat,50.765826,4.279631,moet ervangen wegens toekomstige toename tonna...,technisch,Ja,vermoedelijk >2024,Nee,budget 967.490 euro,NaN,Gemeente,onbekend,onbekend,3
71,Beersel,Vlaanderen,Vlaams-Brabant,Houten voetgangersbrug over de Kesterbeek in H...,50.747655,4.276314,brugdeik reinigen en plaatsen anti-slib,technisch,Ja,2020,Nee,budget 5.800 euro,NaN,Gemeente,nee,onbekend,3


Bruggen die vanwege een technisch of klein onderhoudsprobleem gerenoveerd moeten worden (zoals stabiliteitsproblemen, een brugdek dat gereinigd moet worden, opgehoogd worden, etc) staan vaak in gemeentes. Bruggen die in beheer zijn van gemeentes, zijn vaak kleiner en vereisen dan ook minder vaak groot onderhoud. Twee bruggen komen van de lijst van AWV en staan ook al meteen sinds 2003 op de lijst.

In [331]:
bruggen[bruggen.oorzaak == 'verouderd']

,gemeente,gewest,provincie,brugnaam,latitude,longitude,probleem,oorzaak,werken_gepland,start_werken,bezocht,opmerkingen,beelden,beheerder,hoogproblematisch,lijst_sinds,totaal_gemeente
53,Mechelen,Vlaanderen,Antwerpen,Postzegelbrug (N1) over kanaal Leuven - Dijle,51.016016,4.480411,wordt afgebroken en komen twee bruggen voor in...,verouderd,Ja,2024,Nee,NaN,NaN,Gemeente,ja,onbekend,3
54,Voeren,Vlaanderen,Limburg,"Brug over ""De Gulp"" in Remersdaal",50.743414,5.890580,"is bouwvallig, afgesloten voor gemotoriseerd v...",verouderd,Ja,2020,Nee,NaN,NaN,Gemeente,ja,onbekend,1
58,Hamont-Achel,Vlaanderen,Limburg,Brug Warmbeek,51.254285,5.492588,slijtage,verouderd,Ja,2020,Nee,NaN,NaN,Gemeente,onbekend,onbekend,1
60,Merelbeke,Vlaanderen,Oost-Vlaanderen,Brug over de Ringvaart en R4 bij Fraterstraat,51.005171,3.760860,slijtage,verouderd,Ja,2021,Nee,NaN,NaN,Gemeente,onbekend,onbekend,2
62,Kuurne,Vlaanderen,West-Vlaanderen,Brug over de Heulebeek aan de monding van de Leie,50.847284,3.289945,verouderd,verouderd,Ja,2023,Nee,NaN,NaN,Gemeente,onbekend,onbekend,3
68,Diksmuide,Vlaanderen,West-Vlaanderen,Brug over de Grote Beverdijkvaart ten hoogte v...,51.030010,2.811817,verouderd,verouderd,Ja,2012,Nee,NaN,NaN,Gemeente,onbekend,onbekend,1
69,Beersel,Vlaanderen,Vlaams-Brabant,Brug over de Zenne in de Dworpsestraat,50.765257,4.275173,verouderd,verouderd,Ja,2012,Nee,budget 359.490 euro,NaN,Gemeente,onbekend,onbekend,3
72,Rotselaar,Vlaanderen,Vlaams-Brabant,Brug op de grens van Rotselaar en Tremelo,50.983982,4.735366,verouderd,verouderd,Ja,2021/2022,Nee,"budget wordt verdeeld tussen de gemeentes, bei...",NaN,Gemeente,onbekend,onbekend,1
76,Drogenbos,Vlaanderen,Vlaams-Brabant,Brug in de Drie Fonteinenstraat over de Zenne,50.806132,4.301490,"was lang niet in gebruik, helemaal afgebroken ...",verouderd,Ja,Bezig,Nee,NaN,NaN,Gemeente,onbekend,onbekend,1
77,Bertem,Vlaanderen,Vlaams-Brabant,Brug over de dreef van kerk naar kasteel,50.849629,4.592343,"ouderdom, werken aan de stabiliteit, geen gevaar",verouderd,Ja,dit jaar of volgend jaar,Nee,NaN,NaN,Gemeente,nee,onbekend,1


Verouderde bruggen komen zonder uitzondering bij de gemeente vandaan. Problemen zijn bijvoorbeeld slijtage of lang niet in gebruik geweest. Hier is een interessante notie dat gemeenten misschien vaak geen geld/tijd hebben om bruggen tijdig op te knappen.

In [329]:
bruggen[bruggen.oorzaak == 'groter plan']

,gemeente,gewest,provincie,brugnaam,latitude,longitude,probleem,oorzaak,werken_gepland,start_werken,bezocht,opmerkingen,beelden,beheerder,hoogproblematisch,lijst_sinds,totaal_gemeente
4,Zaventem,Vlaanderen,Vlaams-Brabant,Brug B16 in Hector Henneaulaan over de R0 en R22,50.880841,4.460796,Moet vernieuwd worden bij verbreden R0,groter plan,Ja,2022,Nee,NaN,NaN,AWV,ja,2016,3
5,Zaventem,Vlaanderen,Vlaams-Brabant,"Brug B16, deel oprit naar de R0",50.880902,4.463766,Moet vernieuwd worden bij verbreden R0,groter plan,Ja,2022,Nee,NaN,NaN,AWV,ja,2007,3
8,Zaventem,Vlaanderen,Vlaams-Brabant,Viaduct 9 in de E40 over de R0 in Zaventem (Si...,50.868749,4.469104,Moet vernieuwd worden bij verbreden R0,groter plan,Ja,onbekend,Nee,NaN,NaN,AWV,ja,2009,3
14,Gent,Vlaanderen,Oost-Vlaanderen,Brug over deE40 in de N466,51.038792,3.622478,Wordt vernieuwd met complex Drongen,groter plan,Ja,2021,Ja,Door verkeerssituatie onmogelijk goed te zien,Nee,AWV,ja,1997,24
15,Gent,Vlaanderen,Oost-Vlaanderen,Brug LXXV over de E40,51.041225,3.611991,Wordt vernieuwd met complex Drongen,groter plan,Ja,2021,Ja,"In slechte staat, moeilijk in beeld te brengen",Foto en video,AWV,ja,2001,24
16,Gent,Vlaanderen,Oost-Vlaanderen,Brug LXXIII over de E40 in de Baarleveldstraat,51.036091,3.632100,Wordt vernieuwd met complex Drongen,groter plan,Ja,2021,Ja,"Bouwvallig, mag maar afwisselend in één richti...",Foto en video,AWV,ja,2001,24


De bruggen die onderdeel zijn van een groter plan, zijn allemaal hoogproblematische bruggen van het AWV. Op één brug na staan ze allemaal al langer dan 5 jaar op de lijst, één zelfs al meer dan 20 jaar. Voor allen zijn werken gepland, de meesten voor volgend jaar of het jaar erop. Voor één brug staat 'onbekend' maar omdat deze ook onderdeel is van het vernieuwen bij het verbreden van R0, kunnen we ervan uitgaan dat ook deze in 2022 gerenoveerd zal worden.

In [313]:
bruggen[bruggen.oorzaak == 'ongeval']

,gemeente,gewest,provincie,brugnaam,latitude,longitude,probleem,oorzaak,werken_gepland,start_werken,bezocht,opmerkingen,beelden,beheerder,hoogproblematisch,lijst_sinds,totaal_gemeente
9,Halle,Vlaanderen,Vlaams-Brabant,Brug 1 over de R0 in Drasop in Halle (Buizingen),50.729084,4.270168,Brug afgesloten voor gemotoriseerd verkeer na ...,ongeval,Ja,onbekend,Nee,NaN,NaN,AWV,ja,2018,2
22,Roeselare,Vlaanderen,West-Vlaanderen,Brug B50 N36 Roeselare-Kerkhove in Roeselare (...,50.918747,3.176617,Schade na ongeval,ongeval,Ja,onbekend,Ja,Geen zichtbare schade,Nee,AWV,ja,2018,2
27,Zwevegem,Vlaanderen,West-Vlaanderen,La Flandrebrug,50.816373,3.352322,"Beschadigd na aanrijding, werken in uitvoering",ongeval,Ja,2020,Nee,NaN,NaN,Vlaamse Waterweg,ja,2017,1
33,Gent,Vlaanderen,Oost-Vlaanderen,Beekstraatbrug,51.071344,3.657729,Beschadigd na aanrijding. Werken gepland in 2020,ongeval,Ja,2020,Nee,NaN,NaN,Vlaamse Waterweg,ja,2018,24


De bruggen waar sprake is van een ongeval zijn allemaal redelijk recent op de lijst terecht gekomen. Twee ervan moeten dit jaar nog gerenoveerd worden, van twee is de startdatum voor renovatie nog onbekend, hoewel er wel werken gepland zijn.

## Zijn voor de bruggen al werken gepland?

In [314]:
bruggen[bruggen.hoogproblematisch == 'ja'].werken_gepland.value_counts()

Ja          41
Nee         14
onbekend     1
Name: werken_gepland, dtype: int64

Voor veruit de meeste hoogproblematische bruggen zijn al werken gepland, maar voor enkele nog niet - of onbekend.

In [315]:
bruggen[(bruggen.hoogproblematisch == 'ja') & (bruggen.werken_gepland != 'Ja')].gemeente.unique()

array(['Wommelgem', 'Overrijse', 'Grimbergen', 'Gent', 'Wetteren',
       'Nieuwpoort', 'Kortrijk', 'Merelbeke', 'Sint-Laureins', 'Halle',
       'Anderlecht'], dtype=object)

Dat zijn bruggen in de gemeentes Wommelgem, Overrijse, Halle, Grimbergen, Gent, Wetteren, Nieuwpoort, Kortrijk, Merelbeke, Sint-Laureins en Anderlecht.

In [316]:
bruggen[(bruggen.hoogproblematisch == 'ja') & (bruggen.werken_gepland != 'Ja')].lijst_sinds.value_counts()

2018        4
2019        3
2014        1
1999        1
2010        1
2007        1
2017        1
1997        1
onbekend    1
2013        1
Name: lijst_sinds, dtype: int64

Bijna alle problematische bruggen waarvoor nog geen werken gepland zijn, staan op de lijst van het Vlaams Gewest. Vijf bruggen staan al langer dan vijf jaar op de lijst, de meeste korter (de brug in 1997 zagen we al eerder terug en daarvan weten we dat die waarschijnlijk tegelijk met de andere verbreding over de R0 gerenoveerd gaat worden.)

In [317]:
bruggen[(bruggen.hoogproblematisch != 'ja')].werken_gepland.value_counts()

Ja          46
onbekend    36
Name: werken_gepland, dtype: int64

Voor de overige bruggen die gerenoveerd moet worden, is voor de meerderheid al werken gepland.

In [318]:
bruggen[(bruggen.hoogproblematisch != 'ja')].gewest.value_counts()

Brussels Gewest    44
Vlaanderen         41
Name: gewest, dtype: int64

Van deze bruggen staan de meeste in Brussel: daarvan weten we van bijna geen enkele brug wanneer die gerenoveerd gaat worden, tenzij ze al bezig zijn.

In [319]:
bruggen = bruggen.replace('onbkekend', 'onbekend')

In [320]:
bruggen[(bruggen.hoogproblematisch != 'ja') & (bruggen.werken_gepland == 'Ja')].start_werken.value_counts()

onbekend                    19
Bezig                        8
2012                         2
2023                         2
2021                         2
2020                         2
2019                         2
dit jaar of volgend jaar     1
2022                         1
vermoedelijk >2024           1
10-2020                      1
Aanbesteding                 1
2021/2022                    1
Deskundige aangesteld        1
04-2020                      1
Deze legislatuur             1
Name: start_werken, dtype: int64

In [321]:
bruggen[(bruggen.hoogproblematisch != 'ja') & (bruggen.gewest == 'Vlaanderen')].werken_gepland.value_counts()

Ja    38
Name: werken_gepland, dtype: int64

Voor alle Vlaamse, niet-hoogproblematische bruggen zijn in principe werken gepland.

In [322]:
len(bruggen[(bruggen.werken_gepland == 'Ja') & (bruggen.start_werken == 'onbekend')].index)

24

Er zijn 24 bruggen waarvoor wel werken gepland zijn, maar nog onbekend is wanneer.

In [323]:
bruggen[(bruggen.werken_gepland == 'Ja') & (bruggen.start_werken == 'onbekend')].gemeente.value_counts()

Gent                  6
Mechelen              2
De Pinte              2
Ukkel                 2
Zaventem              1
Elsene                1
Halle                 1
Brugge                1
Antwerpen             1
Kapelle-op-den-Bos    1
Bornem                1
Tienen                1
Heusden-Zolder        1
Roeselare             1
Ardooie               1
Galmaarden            1
Name: gemeente, dtype: int64

Daarvan staan de meeste in Gent.